In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.5/701.5 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [7]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov8/data/') ### without negative images
train_test_split('/content/drive/MyDrive/yolov8/data2/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 1133 images---


  0%|          | 0/907 [00:00<?, ?it/s]

------ Training data created with 80% split 907 images -------


  0%|          | 0/226 [00:00<?, ?it/s]

------ Testing data created with a total of 226 images ----------
------ TASK COMPLETED -------
------ PROCESS STARTED -------
--- This folder has a total number of 1626 images---


  0%|          | 0/1301 [00:00<?, ?it/s]

------ Training data created with 80% split 1301 images -------


  0%|          | 0/325 [00:00<?, ?it/s]

------ Testing data created with a total of 325 images ----------
------ TASK COMPLETED -------


In [8]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.6/78.2 GB disk)


In [1]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/data.yaml epochs=100 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=colors

100% 21.5M/21.5M [00:00<00:00, 201MB/s]
Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/data.yaml, epochs=100, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=colors3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=F

In [2]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/colors/weights/best.pt conf=0.55 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11128293 parameters, 0 gradients, 28.5 GFLOPs

image 1/20 /content/drive/MyDrive/yolov8/test_images/IMG_9225.jpg: 640x480 1 red_car, 77.0ms
image 2/20 /content/drive/MyDrive/yolov8/test_images/IMG_9236.jpg: 640x480 1 black_car, 1 gray_car, 1 red_car, 12.1ms
image 3/20 /content/drive/MyDrive/yolov8/test_images/IMG_9248.jpg: 640x480 1 black_car, 12.2ms
image 4/20 /content/drive/MyDrive/yolov8/test_images/IMG_9249.jpg: 640x480 1 black_car, 12.2ms
image 5/20 /content/drive/MyDrive/yolov8/test_images/IMG_9258.jpg: 640x480 2 black_cars, 1 red_car, 12.2ms
image 6/20 /content/drive/MyDrive/yolov8/test_images/IMG_9259.jpg: 640x480 3 black_cars, 1 red_car, 1 white_car, 12.2ms
image 7/20 /content/drive/MyDrive/yolov8/test_images/IMG_9261.jpg: 640x480 3 black_cars, 1 white_car, 12.2ms
image 8/20 /content/drive/MyDrive/yolov8/test_images/IMG_9271.jpg: 640x480 1 gray_car, 1 white_

In [3]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output

In [4]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/colors/weights/best.pt conf=0.50 source=/content/drive/MyDrive/yolov8/vid2

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11128293 parameters, 0 gradients, 28.5 GFLOPs

video 1/2 (1/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 640x384 1 black_car, 1 white_car, 115.1ms
video 1/2 (2/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 640x384 1 black_car, 1 white_car, 10.8ms
video 1/2 (3/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 640x384 1 black_car, 1 white_car, 10.7ms
video 1/2 (4/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 640x384 1 black_car, 1 white_car, 10.6ms
video 1/2 (5/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 640x384 1 white_car, 10.7ms
video 1/2 (6/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 640x384 1 white_car, 10.6ms
video 1/2 (7/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 640x384 1 black_car, 1 white_car, 10.6ms
video 1/2 (8/658) /content/drive/MyDrive/yolov8/vid2/Download (2).mp4: 64

In [5]:
cp -r /content/runs/detect/predict2 /content/drive/MyDrive/yolov8/output

In [6]:
!yolo mode=export model=/content/drive/MyDrive/yolov8/training_results/colors/weights/best.pt format=onnx

Ultralytics YOLOv8.1.4 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 11128293 parameters, 0 gradients, 28.5 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/yolov8/training_results/colors/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (128.0 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 71.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 9.7s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 11.1s, saved as '/content/drive/MyDrive/yolov8/training_results/colors/weights/best.onnx' (42.7 MB)

Export complete (13.4s)
Results saved to /content/drive/.shortcut-targets-by-id/1i6B1YjJXUCIzR14b8eONi0HJoWT1E5hn/yolov8/training_results/colors/weigh